# Authenticating against workspace

In [ ]:
from azureml.core import Workspace,Dataset,Environment,Experiment,ComputeTarget,ScriptRunConfig

ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

In [ ]:
training_data=Dataset.get_by_name(ws,'Diabetes_Processed')

# Creating environment

### Docs
[Environment class](https://docs.microsoft.com/en-us/python/api/azureml-core/azureml.core.environment(class)?view=azure-ml-py) </br>
[How to docs](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-environments)  </br>



In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
#Creating environment
environment = Environment(name='Diabetes-Model-Environment')
environment.python.conda_dependencies = CondaDependencies.create(
    conda_packages=['pandas','scikit-learn','xgboost','matplotlib']
    )
environment.register(ws)
print("Environment Registered")

# Creating experiment

In [ ]:
#Creating experiment
experiment = Experiment(ws, 'Diabetes-Training')
#Getting compute target
compute_instance = ComputeTarget(workspace=ws, name='EXTNAN1')

# Defining run configuration


In [ ]:
#Defining run config
config = ScriptRunConfig(
        source_directory="./scripts",
        script="./train.py",
        arguments=[
            "--input_data",
            training_data.as_named_input("input_data"),
            "--model_name",
            "Diabetes-Model",
            "--target_column",
            "Binary_Target"
        ],
        compute_target=compute_instance,
        environment=environment,
    )
#Creating run
run = experiment.submit(config=config)
#Output deploy logs to notebook
run.wait_for_completion(show_output=True)
